<a href="https://colab.research.google.com/github/DanielHevdeli/hafifot-tiug/blob/main/LLMs_as_annotators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/DanielHevdeli/hafifot-tiug.git

fatal: destination path 'hafifot-tiug' already exists and is not an empty directory.


In [ ]:
pip install dspy

In [31]:
import pandas as pd
import dspy
from dspy import Signature, Predict
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from typing import Literal

In [5]:
posts_df = pd.read_csv('./hafifot-tiug/data/split_data/present.csv')

In [6]:
posts_df.head()

,question_id,length,date,text
0,114678,903,2016-04-10 20:05:00,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י..."
1,116349,888,2016-04-24 19:56:00,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...
2,108590,1183,2016-02-16 14:23:00,הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים ...
3,100709,2689,2015-12-15 11:33:00,"בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה ק..."
4,120766,1112,2016-05-27 22:26:00,"אני תוהה כבר כמה שנים מה ללמוד בעתיד, איזה מקצ..."


In [7]:
posts_df.iloc[3]['text']

'בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה קיצונית בגיל 18 עקב לימודים ומצב חברתי גרוע והשפלות יומיומיות  . בהתחלה לא הבנתי בכלל שמדובר בחרדה כי הסימנים היו פיזיולוגים והחששות תמיד היו. התחלתי "להתעלף"  , הרגשתי שאני לא מצליחה לנשום וגם כשאני כן מצליחה אז האוויר לא נכנס לי לריאות , היו לי סחרחורות שהייתי בטוחה שאני עומדת למות ולהתמוטט. נשלחתי למיון וכאשר לא מצאו לי כלום הניחו שזה חרדה.ככה העברתי את השנה האחרונה שלי בבית ספר, כיתה יב. כלומר, במיטה ובדיכאון ללא חברים ולומדת לבגרויות. \nכמה חודשים אחר כך עליתי על מדים והאמת שמצבי השתפר פלאים. אומנם החרדה לא נעלמה , ותמיד היו ניצוצות של חרדה אבל הצלחתי לעבור אותן. \nכהשתחררתי היה לי מין תהום ענקית , הרגשתי בחופש ורציתי כל היום לישון,מה שבצבא לא התאפשר לי וגם כן לא לפני . יצאתי מדי פעם עם חברות אבל גם זה אחכ הפסיק. אחרי כמה חודשים כאלה התחלתי את הפסיכומטרי ולצערי בגלל תחושות של חרדה והתמוטטות בשיעור דחיתי את המועד. אחרי הדחייה נכנסתי לדכאון וחרדה קשים , לא רציתי לצאת מהמיטה.אם פעם התקפים באו והלכו אז המצב שלי בחודשיים האחרונים היו 24/7. 

In [8]:
posts_df.iloc[2]['text']

'הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים של אנונימים פה—אני מקווה שגם אוכל למצוא פה מענה או כיוון לפתרון לבעיה שלי.  \nאני לא יודעת כל כך מאיפה להתחיל או מאיפה התחיל הסיפור אבל אני סובלת בערך כל חיי מחרדה ודיכאון. \nהייתי מודעת לעניין אבל מעולם לא חוויתי מצב קיצוני של חרדה משתקת לחלוטין. היו לי התקפים מזעזעים שבגללם הלכתי למיון.  \nאבל בקיצור, בתקופה האחרונה של כמעט שנה אני חוויתי אגרופוביה רצינית עם התקפי דכאון ומחשבות מזעזעות. ברמה של סיעוד. לא התקלחתי, כל היום הייתי ישנה ומסוגרת מתחת לשמיכה. \nהפסיכאטר נתן לי ציפרלקס .  \nאני מרגישה שיפור עם הדיכאון יחסית, ועם ההתקפי חרדה אבל עד היום האגרובופיה לא נעלמה, קשה לי לתפקד. אני רוצה לצאת החוצה,להכיר אנשים, לעבוד, לקבל רישיון, ללמוד. לא להיות למה שהפכתי. \nמיותר לציין,שניתקתי קשר עם כל סבוביי כך שאין לי חיי חברה.  \nאת כל הכסף שאין לי אני מוציאה על מוניות בדרך לפסיכאטר ובחזרה מימנו כי קשה לי לעלות על אוטובוסים. \nהפסקתי עם השיעורי נהיגה בגלל ההתקפים בנהיגה .  \nבקיצור,אם המצב ימשיך אני לא יודעת איך אוכל לשרוד ככה,גם מבחינה כספית וגם 

Let's try to classify each post to either suicidal or non-suicidal. It may help the publishers of the website to offer first-help to writers of post categorized as suicidal even before other people answer them.

# Annotate Posts

In [11]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

In [59]:
class CustomHFModel(dspy.LM):
    def __init__(self, model, tokenizer, model_name_str, device, **kwargs):
        super().__init__(model=model_name_str, **kwargs)
        # Store the actual HuggingFace model object in a separate attribute
        self._hf_model = model
        self.tokenizer = tokenizer
        self.device = device

    # Modify __call__ to accept 'messages' as keyword argument, as passed by dspy's adapters
    def __call__(self, messages=None, **kwargs):
        if messages is None:
            raise ValueError("No messages provided to CustomHFModel.")

        # Convert DSPy's messages format (list of dicts) into a single prompt string
        # using the tokenizer's chat template, if available. Qwen typically uses this.
        formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", return_token_type_ids=False)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Extract common generation parameters, providing defaults if not present in kwargs
        max_tokens = kwargs.pop("max_tokens", 150)
        temperature = kwargs.pop("temperature", 0.0)
        do_sample = kwargs.pop("do_sample", False)
        stop_tokens = kwargs.pop("stop_tokens", None)

        outputs = self._hf_model.generate( # Use the stored actual HuggingFace model
            **inputs,
            max_new_tokens=max_tokens,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
            do_sample=do_sample,
            temperature=temperature,
            **kwargs, # Pass any remaining kwargs to the generate function
        )

        # Decode only the newly generated tokens
        generated_text = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        # Handle stop tokens
        if stop_tokens:
            for stop_token in stop_tokens:
                if stop_token in generated_text:
                    generated_text = generated_text.split(stop_token)[0]

        return [generated_text]

In [ ]:
lm = CustomHFModel(model=model, tokenizer=tokenizer, model_name_str=MODEL_NAME, device=device)
dspy.configure(lm=lm)

print(f"dspy configured with {MODEL_NAME}")

In [42]:
class Classify(dspy.Signature):
    """Classify label of a given post."""

    sentence: str = dspy.InputField()
    label: Literal["suicidal", "non-suicidal"] = dspy.OutputField()

classify = dspy.Predict(Classify)

In [61]:
classified_posts = []
i = 0
for index, row in posts_df.iterrows():
    i += 1
    question_id = row['question_id']
    text = row['text']

    # Use the classify function
    prediction = classify(sentence=text)
    label = prediction.label

    # Store the results
    classified_posts.append({'question_id': question_id, 'label': label})
    if i >=100:
      break
print("Classification complete. The first 5 classified posts are:")
print(classified_posts[:5])

Classification complete. The first 5 classified posts are:
[{'question_id': 114678, 'label': 'non-suicidal'}, {'question_id': 116349, 'label': 'suicidal'}, {'question_id': 108590, 'label': 'suicidal'}, {'question_id': 100709, 'label': 'non-suicidal'}, {'question_id': 120766, 'label': 'non-suicidal'}]


In [62]:
classified_df = pd.DataFrame(classified_posts)
classified_df.to_csv('hafifot-tiug/data/labels/present_qwen2_labels.csv', index=False)
print("Classified posts saved")

Classified posts saved


In [63]:
labels_df = pd.read_csv('hafifot-tiug/data/labels/present_qwen2_labels.csv')

In [64]:
labels_df.head(10)

,question_id,label
0,114678,non-suicidal
1,116349,suicidal
2,108590,suicidal
3,100709,non-suicidal
4,120766,non-suicidal
5,126063,suicidal
6,102524,suicidal
7,113320,suicidal
8,105981,non-suicidal
9,120977,suicidal
